In [5]:
import argparse
import geopandas as gpd
import numpy as np
import pickle
from functools import partial
from gerrychain import Graph, GeographicPartition, Partition, Election, accept
from gerrychain.updaters import Tally, cut_edges
from gerrychain import MarkovChain
from gerrychain.proposals import recom
from gerrychain.accept import always_accept
from gerrychain import constraints
from gerrychain.tree import recursive_tree_part
from little_helpers import *
import json

In [6]:
def config_markov_chain(initial_part, iters=1000, epsilon=0.05, 
                        compactness=True, pop="TOT_POP", accept_func=None):
    ideal_population = np.nansum(list(initial_part["population"].values())) / len(initial_part)

    proposal = partial(recom,
                       pop_col=pop,
                       pop_target=ideal_population,
                       epsilon=epsilon,
                       node_repeats=1)

    if compactness:
        compactness_bound = constraints.UpperBound(lambda p: len(p["cut_edges"]),
                            2*len(initial_part["cut_edges"]))
        cs = [constraints.within_percent_of_ideal_population(initial_part, epsilon),
              compactness_bound]
    else:
        cs = [constraints.within_percent_of_ideal_population(initial_part, epsilon)]


    if accept_func == None: accept_func = accept.always_accept

    return MarkovChain(proposal=proposal, constraints=cs,
                       accept=accept_func, initial_state=initial_part,
                       total_steps=iters)


In [45]:
class Gingleator:
    """
    Gingleator class

    This class represents a set of methods used to find plans with greater numbers
    of gingles districts.
    """

    def __init__(self, initial_partition, threshold=0.4, 
                 score_funct=None, minority_perc_col=None,
                 pop_col="TOTPOP", epsilon=0.05):
        self.part = initial_partition
        self.threshold = threshold
        self.score = self.num_opportunity_dists if score_funct == None else score_funct
        self.minority_perc = minority_perc_col
        self.pop_col = pop_col
        self.epsilon = epsilon


    def init_minority_perc_col(self, minority_pop_col, total_pop_col,
                               minority_perc_col):
        """
         init_minority_perc_col takes the string corresponding to the minority
         population column and the total population column attributes in the 
         partition updaters as well as the desired name of the minority percent 
         column and updates the partition updaters accordingly
        """
        perc_up = {minority_perc_col:
                   lambda part: {k: part[minority_pop_col][k] / part[total_pop_col][k]
                                 for k in part.parts.keys()}}
        self.part.updaters.update(perc_up)


    """
    Types of Markov Chains:
    The following methods are different strategies for searching for the maximal
    number of Gingles districts
    """

    def short_burst_run(self, num_bursts, num_steps, verbose=False,
                        maximize=True, tracking_fun=None): #checkpoint_file=None):
        max_part = (self.part, self.score(self.part, self.minority_perc,
                    self.threshold)) 
        """
        short_burst_run: preforms a short burst run using the instance's score function.
                         Each burst starts at the best preforming plan of the previous
                         burst.  If there's a tie, the later observed one is selected.
        args:
            num_steps:  how many steps to run an unbiased markov chain for during each burst
            num_bursts: how many bursts to preform
            verbose:    flag - indicates whether to prints the burst number at the beginning of 
                               each burst
            maximize:   flag - indicates where to prefer plans with higher or lower scores.
            tracking_fun: Function to save information about each observed plan.
        """
        observed_num_ops = np.zeros((num_bursts, num_steps))

        for i in range(num_bursts):
            if verbose: print("*", end="", flush=True)
            chain = config_markov_chain(max_part[0], iters=num_steps,
                                        epsilon=self.epsilon, pop=self.pop_col)

            for j, part in enumerate(chain):
                part_score = self.score(part, self.minority_perc, self.threshold)
                observed_num_ops[i][j] = part_score
                if maximize:
                    max_part = (part, part_score) if part_score >= max_part[1] else max_part
                else:
                    max_part = (part, part_score) if part_score <= max_part[1] else max_part

                if tracking_fun != None: tracking_fun(part, i, j)

        return (max_part, observed_num_ops)


    def variable_len_short_burst(self, num_iters, stuck_buffer=10,
                                 maximize=True, verbose=False):
        """
        variable_len_short_burst: preforms a variable length short burst run using the instance's 
                                  score function. Each burst starts at the best preforming plan of 
                                  the previous burst.  If there's a tie, the later observed one is 
                                  selected.
        args:
            num_iters:      the total number of steps to take (aka plans to sample)
            stuck_buffer:   Factor specifying how long to tolerate no improvement, before increasing
                            the burst length.
            verbose:        flag - indicates whether to prints the burst number at the beginning 
                                    of each burst
            maximize:       flag - indicates where to prefer plans with higher or lower scores.
        """
        max_part = (self.part, self.score(self.part, self.minority_perc,
                        self.threshold))
        observed_num_ops = np.zeros(num_iters)
        time_stuck = 0
        burst_len = 2
        i = 0

        while(i < num_iters):
            if verbose: print("*", end="", flush=True)
            chain = config_markov_chain(max_part[0], iters=burst_len,
                                        epsilon=self.epsilon, pop=self.pop_col)
            for j, part in enumerate(chain):
                part_score = self.score(part, self.minority_perc, self.threshold)
                observed_num_ops[i] = part_score

                if part_score <= max_part[1]: time_stuck += 1
                else: time_stuck = 0

                if maximize:
                    max_part = (part, part_score) if part_score >= max_part[1] else max_part
                else:
                    max_part = (part, part_score) if part_score <= max_part[1] else max_part
                
                i += 1
                if i >= num_iters: break
            if time_stuck >= stuck_buffer*burst_len : burst_len *= 2

        return (max_part, observed_num_ops)


    def biased_run(self, num_iters, p=0.25, maximize=True, verbose=False):
        """
        biased_run: preforms a biased (or tilted) run using the instance's score function.  The
                    chain always accepts a new proposal with the same or a better score and accepts
                    proposals with a worse score with some probability.
        args:
            num_iters:  total number of steps to take (aka plans to sample)
            p:          probability of a plan with a worse preforming score
            verbose:    flag - indicates whether to prints the burst number at the beginning 
                                    of each burst
            maximize:   flag - indicates where to prefer plans with higher or lower scores.
        """
        max_part = (self.part, self.score(self.part, self.minority_perc,
                    self.threshold))
        observed_num_ops = np.zeros(num_iters)
        
        def biased_acceptance_function(part):
            if part.parent == None: return True
            part_score = self.score(part, self.minority_perc, self.threshold)
            prev_score = self.score(part.parent, self.minority_perc, self.threshold)
            if maximize and part_score >= prev_score: return True
            elif not maximize and part_score <= prev_score: return True
            else: return random.random() < p

        chain = config_markov_chain(self.part, iters=num_iters,
                                    epsilon=self.epsilon, pop=self.pop_col,
                                    accept_func= biased_acceptance_function)
        for i, part in enumerate(chain):
            if verbose and i % 100 == 0: print("*", end="", flush=True)
            part_score = self.score(part, self.minority_perc, self.threshold)
            observed_num_ops[i] = part_score
            if maximize:
                max_part = (part, part_score) if part_score >= max_part[1] else max_part
            else:
                max_part = (part, part_score) if part_score <= max_part[1] else max_part

        return (max_part, observed_num_ops)


    def biased_short_burst_run(self, num_bursts, num_steps, p=0.25, 
                              verbose=False, maximize=True):
        """
        biased_short_burst_run: preforms a biased short burst run using the instance's score function.
                                Each burst is a biased run markov chain, starting at the best preforming 
                                plan of the previous burst.  If there's a tie, the later observed 
                                one is selected.
        args:
            num_steps:  how many steps to run an unbiased markov chain for during each burst
            num_bursts: how many bursts to preform
            p:          probability of a plan with a worse preforming score, within a burst
            verbose:    flag - indicates whether to prints the burst number at the beginning of 
                               each burst
            maximize:   flag - indicates where to prefer plans with higher or lower scores.
        """
        max_part = (self.part, self.score(self.part, self.minority_perc,
                    self.threshold)) 
        observed_num_ops = np.zeros((num_bursts, num_steps))

        def biased_acceptance_function(part):
            if part.parent == None: return True
            part_score = self.score(part, self.minority_perc, self.threshold)
            prev_score = self.score(part.parent, self.minority_perc, self.threshold)
            if maximize and part_score >= prev_score: return True
            elif not maximize and part_score <= prev_score: return True
            else: return random.random() < p

        for i in range(num_bursts):
            if verbose: print("Burst:", i)
            chain = config_markov_chain(max_part[0], iters=num_steps,
                                        epsilon=self.epsilon, pop=self.pop_col,
                                        accept_func= biased_acceptance_function)

            for j, part in enumerate(chain):
                part_score = self.score(part, self.minority_perc, self.threshold)
                observed_num_ops[i][j] = part_score
                if maximize:
                    max_part = (part, part_score) if part_score >= max_part[1] else max_part
                else:
                    max_part = (part, part_score) if part_score <= max_part[1] else max_part
    
        return (max_part, observed_num_ops)

    """
    Score Functions
    """

    @classmethod
    def num_opportunity_dists(cls, part, minority_perc, threshold):
        """
        num_opportunity_dists: given a partition, name of the minority percent updater, and a
                               threshold, returns the number of opportunity districts.
        """
        dist_precs = part[minority_perc].values()
        return sum(list(map(lambda v: v >= threshold, dist_precs)))


    @classmethod
    def reward_partial_dist(cls, part, minority_perc, threshold):
        """
        reward_partial_dist: given a partition, name of the minority percent updater, and a
                             threshold, returns the number of opportunity districts + the 
                             percentage of the next highest district.
        """
        dist_percs = part[minority_perc].values()
        num_opport_dists = sum(list(map(lambda v: v >= threshold, dist_percs)))
        next_dist = max(i for i in dist_percs if i < threshold)
        return num_opport_dists + next_dist


    @classmethod
    def reward_next_highest_close(cls, part, minority_perc, threshold):
        """
        reward_next_highest_close: given a partition, name of the minority percent updater, and a
                                   threshold, returns the number of opportunity districts, if no 
                                   additional district is within 10% of reaching the threshold.  If one is, 
                                   the distance that district is from the threshold is scaled between 0 
                                   and 1 and added to the count of opportunity districts.
        """
        dist_precs = part[minority_perc].values()
        num_opport_dists = sum(list(map(lambda v: v >= threshold, dist_precs)))
        next_dist = max(i for i in dist_precs if i < threshold)

        if next_dist < threshold - 0.1:
            return num_opport_dists
        else: 
            return num_opport_dists + (next_dist - threshold + 0.1)*10


    @classmethod
    def penalize_maximum_over(cls, part, minority_perc, threshold):
        """
        penalize_maximum_over: given a partition, name of the minority percent updater, and a
                               threshold, returns the number of opportunity districts + 
                               (1 - the maximum excess) scaled to between 0 and 1.
        """
        dist_precs = part[minority_perc].values()
        num_opportunity_dists = sum(list(map(lambda v: v >= threshold, dist_precs)))
        if num_opportunity_dists == 0:
            return 0
        else:
            max_dist = max(dist_precs)
            return num_opportunity_dists + (1 - max_dist)/(1 - threshold)


    @classmethod
    def penalize_avg_over(cls, part, minority_perc, threshold):
        """
        penalize_maximum_over: given a partition, name of the minority percent updater, and a
                               threshold, returns the number of opportunity districts + 
                               (1 - the average excess) scaled to between 0 and 1.
        """
        dist_precs = part[minority_perc].values()
        opport_dists = list(filter(lambda v: v >= threshold, dist_precs))
        if opport_dists == []:
            return 0
        else:
            num_opportunity_dists = len(opport_dists)
            avg_opportunity_dist = np.mean(opport_dists)
            return num_opportunity_dists + (1 - avg_opportunity_dist)/(1 - threshold)

In [8]:
score_functs = {0: None, 1: Gingleator.reward_partial_dist, 
                2: Gingleator.reward_next_highest_close,
                3: Gingleator.penalize_maximum_over,
                4: Gingleator.penalize_avg_over}

In [36]:
BURST_LEN = 10
NUM_DISTRICTS = 7
ITERS = 1000
POP_COL = "TOTPOP"
N_SAMPS = 10
SCORE_FUNCT = None
EPS = 0.01
MIN_POP_COL = "BVAP"

In [37]:
## Setup graph, updaters, elections, and initial partition

print("Reading in Data/Graph", flush=True)
sc_gdf = gpd.read_file("./shapefiles/SC.geojson")
sc_graph = Graph.from_geodataframe(sc_gdf)

Reading in Data/Graph


In [38]:
my_updaters = {"population" : Tally(POP_COL, alias="population"),
               "VAP": Tally("VAP"),
               "BVAP": Tally("BVAP"),
               "nBVAP": lambda p: {k: v - p["BVAP"][k] for k,v in p["VAP"].items()},
               "cut_edges": cut_edges}

In [39]:
initial_plan = {}
for node, data in sc_graph.nodes(data=True):
    if 'CD' in data:
        initial_plan[node] = data['CD']
print("Congressional District Plan:", initial_plan)

Congressional District Plan: {0: '1', 1: '1', 2: '1', 3: '1', 4: '1', 5: '1', 6: '1', 7: '1', 8: '1', 9: '1', 10: '1', 11: '1', 12: '1', 13: '1', 14: '1', 15: '1', 16: '1', 17: '1', 18: '1', 19: '1', 20: '1', 21: '1', 22: '1', 23: '1', 24: '1', 25: '1', 26: '1', 27: '1', 28: '1', 29: '1', 30: '1', 31: '1', 32: '1', 33: '1', 34: '1', 35: '1', 36: '1', 37: '1', 38: '1', 39: '1', 40: '1', 41: '1', 42: '1', 43: '1', 44: '1', 45: '1', 46: '1', 47: '1', 48: '1', 49: '1', 50: '1', 51: '1', 52: '1', 53: '1', 54: '1', 55: '1', 56: '1', 57: '1', 58: '1', 59: '1', 60: '1', 61: '1', 62: '1', 63: '1', 64: '1', 65: '1', 66: '1', 67: '1', 68: '1', 69: '1', 70: '1', 71: '1', 72: '1', 73: '1', 74: '1', 75: '1', 76: '1', 77: '1', 78: '1', 79: '1', 80: '1', 81: '1', 82: '1', 83: '1', 84: '1', 85: '1', 86: '1', 87: '1', 88: '1', 89: '1', 90: '1', 91: '1', 92: '1', 93: '2', 94: '2', 95: '2', 96: '2', 97: '2', 98: '2', 99: '2', 100: '2', 101: '2', 102: '2', 103: '2', 104: '2', 105: '2', 106: '2', 107: '2', 

In [40]:
init_partition = Partition(sc_graph, assignment=initial_plan, updaters=my_updaters)

In [46]:
gingles = Gingleator(init_partition, pop_col=POP_COL,
                     threshold=0.5, score_funct=SCORE_FUNCT, epsilon=EPS,
                     minority_perc_col="BVAP_perc")

gingles.init_minority_perc_col("BVAP", "VAP", 
                               "BVAP_perc")

num_bursts = int(ITERS/BURST_LEN)

Starting Short Bursts Runs


In [42]:
print("Starting Short Bursts Runs", flush=True)
for n in range(N_SAMPS):
    sb_obs = gingles.short_burst_run(num_bursts=num_bursts, num_steps=BURST_LEN,
                                     maximize=True, verbose=False)
    print("\tFinished chain {}".format(n), flush=True)

    print("\tSaving results", flush=True)

    f_out = "data/SC_dists{}_{}opt_{:.1%}_{}_sbl{}_score{}_{}.npy".format(NUM_DISTRICTS, "BVAP", EPS, 
                                                        ITERS, BURST_LEN, "0", n)
    np.save(f_out, sb_obs[1])

    f_out_part = "data/SC_dists{}_{}opt_{:.1%}_{}_sbl{}_score{}_{}_max_part.p".format(NUM_DISTRICTS, "BVAP", EPS, 
                                                        ITERS, BURST_LEN, "0", n)

    max_stats = {"VAP": sb_obs[0][0]["VAP"],
                 "BVAP": sb_obs[0][0]["BVAP"]}

    with open(f_out_part, "wb") as f_out:
        pickle.dump(max_stats, f_out)

	Finished chain 0
	Saving results
	Finished chain 1
	Saving results
	Finished chain 2
	Saving results
	Finished chain 3
	Saving results
	Finished chain 4
	Saving results
	Finished chain 5
	Saving results
	Finished chain 6
	Saving results
	Finished chain 7
	Saving results
	Finished chain 8
	Saving results
	Finished chain 9
	Saving results


In [47]:
print("Starting Tilted Runs", flush=True)
P = 0.125
for n in range(N_SAMPS):
    tilt_obs = gingles.biased_run(num_iters=ITERS, p=P, maximize=True, verbose=True)
    print("\n\tFinished chain {}".format(n), flush=True)

    print("\tSaving results", flush=True)

    f_out = "data/SC_dists{}_{}opt_{:.1%}_{}_p{}_{}.npy".format(
                                                        NUM_DISTRICTS,"BVAP", EPS, 
                                                        ITERS, P, n)
    np.save(f_out, tilt_obs[1])

    f_out_part = "data/SC_dists{}_{}opt_{:.1%}_{}_p{}_{}_max_part.p".format(
                                                        NUM_DISTRICTS,"BVAP", EPS, 
                                                        ITERS, P, n)

    max_stats = {"VAP": tilt_obs[0][0]["VAP"],
                 "BVAP": tilt_obs[0][0]["BVAP"]}

    with open(f_out_part, "wb") as f_out:
        pickle.dump(max_stats, f_out)

Starting Tilted Runs
**********
	Finished chain 0
	Saving results
**********
	Finished chain 1
	Saving results
**********
	Finished chain 2
	Saving results
**********
	Finished chain 3
	Saving results
**********
	Finished chain 4
	Saving results
**********
	Finished chain 5
	Saving results
**********
	Finished chain 6
	Saving results
**********
	Finished chain 7
	Saving results
**********
	Finished chain 8
	Saving results
**********
	Finished chain 9
	Saving results


In [52]:
## Setup chain

total_pop = sum([sc_graph.nodes()[n][POP_COL] for n in sc_graph.nodes()])
ideal_pop = total_pop / NUM_DISTRICTS

print(total_pop, ideal_pop)

5118425 731203.5714285715


In [53]:
proposal = partial(recom, pop_col=POP_COL, pop_target=ideal_pop, epsilon=EPS, 
                   node_repeats=1)

compactness_bound = constraints.UpperBound(lambda p: len(p["cut_edges"]), 
                                             2*len(init_partition["cut_edges"]))

chain = MarkovChain(
        proposal,
        constraints=[
            constraints.within_percent_of_ideal_population(init_partition, EPS),
            compactness_bound],
        accept=accept.always_accept,
        initial_state=init_partition,
        total_steps=ITERS)

In [54]:
print("Starting Markov Chain runs")

for n in range(N_SAMPS):
    print("\tStarting chain {}".format(n), flush=True)
    chain_results = {"cutedges": np.zeros(ITERS),
                     "BVAP": np.zeros((ITERS, NUM_DISTRICTS))}

    output = "data/unbiased_SC_dists{}_{:.1%}_{}_unbiased_{}.p".format(NUM_DISTRICTS, EPS, 
                                                        ITERS, n)


    for i, part in enumerate(chain):
        chain_results["cutedges"][i] = len(part["cut_edges"])
        chain_results["BVAP"][i] = sorted([v / part["VAP"][k] for k,v in part["BVAP"].items()])

        
        if i % 1000 == 0:
            print("*", end="", flush=True)

    with open(output, "wb") as f_out:
        pickle.dump(chain_results, f_out)

    print()

Starting Markov Chain runs
	Starting chain 0
*
	Starting chain 1
*
	Starting chain 2
*
	Starting chain 3
*
	Starting chain 4
*
	Starting chain 5
*
	Starting chain 6
*
	Starting chain 7
*
	Starting chain 8
*
	Starting chain 9
*
